In [534]:
import pandas as pd
import numpy as np
import re

In [535]:
path = '../data/_data.csv'
df = pd.read_csv(path)
pd.set_option('display.max_columns', None)

In [536]:
df.drop(['Unnamed: 0','Тип', 'Телефоны', 
        #  'Ссылка на объявление',
           'Серия дома' ], axis= 1, inplace=True)


Площадь комнат

In [538]:
#Функция для приведения столбца "Площадь комнат, м2" к одному числу - суммарной площади комнат
def unify_room_square(row):
    if pd.isna(row['Площадь комнат, м2']):
        return np.nan

    parts = re.findall(r'\d+', row['Площадь комнат, м2'])

    parts = [float(part) for part in parts]
    
    # Получаем количество чисел в столбце "Площадь, м2"
    area_split = row['Площадь, м2'].split('/')
    second_area = float(area_split[1]) if len(area_split) == 3 else np.nan
    
    # Проверяем, равняется ли сумма чисел в строке второму числу из "Площадь, м2"
    if sum(parts) != second_area and not pd.isna(second_area):
        return second_area
    
    return sum(parts)

df['Площадь комнат, м2'] = df.apply(unify_room_square, axis=1)

In [539]:
#Количество пропусков по площади комнат
df['Площадь комнат, м2'].isna().value_counts()

Площадь комнат, м2
False    14458
True      8910
Name: count, dtype: int64

In [540]:
#Функция для заполнения пропусков в столбце "Площадь комнат, м2" к одному числу - суммарной площади комнат
def notnan_room_square(row):
    if pd.isna(row['Площадь комнат, м2']):
        area_split = row['Площадь, м2'].split('/')

        if len(area_split) == 3:
            return float(area_split[1])
        
        if len(area_split) == 2:
            return float(area_split[0]) - float(area_split[1]) - 4  #4 - средняя площадь санузла
        
        if len(area_split) == 1:
            return float(area_split[0]) - 4 - 10   #10 - средняя площадь кухни

    else:
        return row['Площадь комнат, м2']

df['Площадь комнат, м2'] = df.apply(notnan_room_square, axis=1)

In [541]:
df['Площадь комнат, м2'].isna().value_counts()
#Пропусков нет

Площадь комнат, м2
False    23368
Name: count, dtype: int64

Балкон

In [542]:
#Количество пропусков в столбце с Балконом
df['Балкон'].isna().value_counts()

Балкон
False    15390
True      7978
Name: count, dtype: int64

In [543]:
df['Балкон'].value_counts()

Балкон
Балкон (1)                7428
Лоджия (1)                6007
Балкон (1), Лоджия (1)     716
Лоджия (2)                 568
Балкон (2)                 474
Балкон (3)                  55
Лоджия (3)                  45
Балкон (2), Лоджия (2)      25
Балкон (1), Лоджия (2)      24
Балкон (2), Лоджия (1)      20
Балкон (4)                   6
Балкон (3), Лоджия (1)       5
Балкон (1), Лоджия (3)       5
Лоджия (4)                   5
Балкон (2), Лоджия (3)       3
Балкон (1), Лоджия (4)       2
Балкон (3), Лоджия (3)       1
Балкон (4), Лоджия (4)       1
Name: count, dtype: int64

In [544]:
#Функция для заполнения пропусков в столбце Балкон на самые часто встречающиеся значение - Балкон(1) и Лоджия(1)
def notna_balcony(row):
    if pd.isna(row['Балкон']): 
        floor = int(re.findall(r'\d+', row['Дом'])[0])  # Получаем первый найденный этаж
        type_of_house = row['Дом'].split(',')[-1].strip()  # Убираем лишние пробелы
        area = float(row['Площадь, м2'].split('/')[0])
        if floor > 3 and area > 100 and type_of_house != 'Монолитный':
            return 'Балкон (1)'
        else:
            return 'Лоджия (1)'
    else:
        return row['Балкон']
    
df['Балкон'] = df.apply(notna_balcony, axis=1)

Окна

In [545]:
#Сколько раз встречаются уникальные значения в столбце Окна
df['Окна'].value_counts()

Окна
Во двор            10870
На улицу и двор     3295
На улицу            2590
Name: count, dtype: int64

In [546]:
#Количество пропусков в столбце Окна
df['Окна'].isna().value_counts()

Окна
False    16755
True      6613
Name: count, dtype: int64

In [547]:
#Согласно анализу объявлений, если в описании присутствует фраза "вид на", то, как правило вид из окон на достопримечательность, что сыграет роль в стоимости аренды
#Функция для заполнения значений по описанию:
def from_description(row):
    if pd.isna(row['Окна']) and 'вид на' in row['Описание'].lower():
        return 'На достопримечательность'
    
    elif pd.isna(row['Окна']) and 'вид во' in row['Описание'].lower():
        return 'Во двор'
    else:
        return row['Окна']

df['Окна'] = df.apply(from_description, axis=1)

In [549]:
df['Окна'].value_counts()

Окна
Во двор                     10878
На улицу и двор              3295
На улицу                     2590
На достопримечательность      210
Name: count, dtype: int64

In [550]:
#Функция для заполнения пропусков в столбце Окна в зависимости от этажа
def from_floor(row):
    if pd.isna(row['Окна']): 
        floor = int(re.findall(r'\d+', row['Дом'])[0])
        if floor < 6:
            return 'Во двор'
    else:
        return row['Окна']    
 
df['Окна'] = df.apply(from_floor, axis=1) 

In [551]:
#Окончательно заполняем пропуски:

# Вычисляем частоту встречания каждого значения в столбце "Окна"
window_counts = df['Окна'].value_counts(normalize=True)

# Заполняем пропуски в столбце "Окна" значениями на основе их частоты встречания
df['Окна'].fillna(pd.Series(np.random.choice(window_counts.index, p=window_counts, size=len(df))), inplace=True)

/tmp/ipykernel_45997/3820324534.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Окна'].fillna(pd.Series(np.random.choice(window_counts.index, p=window_counts, size=len(df))), inplace=True)


In [552]:
df['Окна'].value_counts()

Окна
Во двор                     16308
На улицу и двор              3801
На улицу                     3011
На достопримечательность      248
Name: count, dtype: int64

Санузел

In [553]:
df['Санузел'].value_counts()

Санузел
Совмещенный (1)                    10078
Раздельный (1)                      7158
Совмещенный (2)                     1437
Совмещенный (1), Раздельный (1)      812
Раздельный (2)                       534
Совмещенный (3)                      241
Совмещенный (2), Раздельный (1)      188
Совмещенный (4)                       77
Раздельный (3)                        52
Совмещенный (1), Раздельный (2)       30
Совмещенный (3), Раздельный (1)       27
Совмещенный (2), Раздельный (2)       25
Раздельный (4)                        15
Совмещенный (3), Раздельный (3)        6
Совмещенный (4), Раздельный (1)        6
Совмещенный (4), Раздельный (2)        4
Совмещенный (1), Раздельный (3)        2
Совмещенный (2), Раздельный (3)        2
Совмещенный (2), Раздельный (4)        1
Совмещенный (3), Раздельный (2)        1
Name: count, dtype: int64

In [554]:
#Заполним пропуски модой при группировке по количеству комнат:
mode_per_room = df.groupby('Количество комнат')['Санузел'].transform(lambda x: x.mode().iloc[0])
df['Санузел'].fillna(mode_per_room, inplace=True)

/tmp/ipykernel_45997/1272927668.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Санузел'].fillna(mode_per_room, inplace=True)


In [555]:
df.columns

Index(['ID  объявления', 'Количество комнат', 'Метро', 'Адрес', 'Площадь, м2',
       'Дом', 'Парковка', 'Цена', 'Описание', 'Ремонт', 'Площадь комнат, м2',
       'Балкон', 'Окна', 'Санузел', 'Можно с детьми/животными',
       'Дополнительно', 'Название ЖК', 'Высота потолков, м', 'Лифт',
       'Мусоропровод', 'Ссылка на объявление'],
      dtype='object')

Можно с детьми/животными

In [556]:
df['Можно с детьми/животными'].isna().value_counts()

Можно с детьми/животными
False    17272
True      6096
Name: count, dtype: int64

In [557]:
df['Можно с детьми/животными'].value_counts()

Можно с детьми/животными
Можно с детьми                       10134
Можно с детьми, Можно с животными     6899
Можно с животными                      239
Name: count, dtype: int64

In [558]:
# Применяем функцию для заполнения пропусков в столбце "Можно с детьми/животными" на основе описания
def fill_children_animals_description(row):
    if pd.isna(row['Можно с детьми/животными']):
        description = row['Описание'].lower()
        if any(word in description for word in ['детск', 'ребен', 'школ', 'семь']):
            if 'нельзя с' not in description:
                return 'Можно с детьми'
            else:
                return 'Нельзя никого'
        elif any(word in description for word in ['питом', 'животн', 'собак', 'кошк', 'пес', 'кот']):
            return 'Можно с животными'
        elif 'нельзя с' in description:
            return 'Нельзя никого'
        else:
            return np.nan
    else:
        return row['Можно с детьми/животными']

df['Можно с детьми/животными'] = df.apply(fill_children_animals_description, axis=1)

In [559]:
# Применяем функцию для заполнения пропусков в столбце "Можно с детьми/животными" на основе площади квартиры
def fill_children_animals_area(row):
    if pd.isna(row['Можно с детьми/животными']):
        if float(row['Площадь, м2'].split('/')[0]) < 50:
            return 'Нельзя никого'
        else:
            return 'Можно с детьми'   #Оставшиеся значения заполняются самым часто встречающимся
    else:
        return row['Можно с детьми/животными']   
            

df['Можно с детьми/животными'] = df.apply(fill_children_animals_area, axis=1)

Дополнительно

In [560]:
df['Дополнительно'].value_counts()

Дополнительно
Мебель в комнатах, Мебель на кухне, Ванна, Стиральная машина, Телевизор, Холодильник, Интернет                                       2896
Мебель в комнатах, Мебель на кухне, Ванна, Стиральная машина, Холодильник, Интернет                                                  1318
Мебель в комнатах, Мебель на кухне, Ванна, Стиральная машина, Кондиционер, Посудомоечная машина, Телевизор, Холодильник, Интернет    1042
Мебель в комнатах, Мебель на кухне, Ванна, Стиральная машина, Кондиционер, Телевизор, Холодильник, Интернет                           967
Мебель в комнатах, Мебель на кухне, Ванна, Стиральная машина, Телевизор, Холодильник, Интернет, Телефон                               887
                                                                                                                                     ... 
Мебель в комнатах, Душевая кабина, Стиральная машина, Холодильник, Интернет                                                             1
Мебель на кухне, Ван

In [561]:
df['Дополнительно'].isna().value_counts()

Дополнительно
False    23011
True       357
Name: count, dtype: int64

In [562]:
df[df['Описание'].str.contains('без мебели', case=False, na=False)]

,ID объявления,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
8,273973191,3,м. Смоленская (9 мин пешком),"Москва, Новинский бульвар, 18С1",120.0/95.0/10.0,"5/10, Сталинский",открытая,"130000.0 руб./ За месяц, Залог - 130000 руб., ...",Лот 71833. Евгений Николаев.\n\nБонус агенту 1...,Евроремонт,95.0,Балкон (1),На улицу,Совмещенный (1),Можно с животными,"Мебель на кухне, Ванна, Стиральная машина, Кон...",NaN,3.0,Пасс (1),Нет,https://www.cian.ru/rent/flat/273973191
19,274748017,"5, Оба варианта",м. Кропоткинская (6 мин пешком),"Москва, Филипповский переулок, 9",201.0/162.0/16.0,3/3,NaN,"190000.0 руб./ За месяц, Залог - 190000 руб., ...",ID 10903: Просторная 5-комнатная квартира в д...,Евроремонт,162.0,Лоджия (1),На улицу и двор,Совмещенный (3),"Можно с детьми, Можно с животными","Мебель на кухне, Ванна, Душевая кабина, Стирал...",NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/274748017
56,274617827,5,м. Кропоткинская (8 мин пешком),"Москва, Филипповский переулок, 9",200.0/150.0/30.0,"3/3, Кирпичный",NaN,"190000.0 руб./ За месяц, Залог - 190000 руб., ...",Предлагается 5-ти комнатная квартира в отреста...,Евроремонт,150.0,Лоджия (1),Во двор,Совмещенный (2),"Можно с детьми, Можно с животными","Мебель на кухне, Стиральная машина, Телевизор,...",NaN,NaN,NaN,NaN,https://www.cian.ru/rent/flat/274617827
69,271264562,5,м. Кропоткинская (5 мин пешком),"Москва, Гоголевский бульвар, 23",170.0/104.0/21.0,"5/5, старый фонд",наземная,"175000.0 руб./ За месяц, Залог - 175000 руб., ...",Лот 67428. Анастасия Баталова.\n\nБонус агенту...,Евроремонт,104.0,Балкон (1),На улицу и двор,Совмещенный (2),"Можно с детьми, Можно с животными","Мебель на кухне, Душевая кабина, Стиральная ма...",NaN,3.2,Пасс (1),Да,https://www.cian.ru/rent/flat/271264562
74,271766412,6,м. Александровский сад (4 мин пешком),"Москва, переулок Романов, 5",228.0/159.0/30.0,3/6,NaN,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",Лот 30273. Коллегам бонус 50 000 рублей. Татья...,Евроремонт,159.0,Балкон (1),Во двор,Совмещенный (2),Можно с детьми,"Мебель на кухне, Душевая кабина, Стиральная ма...",NaN,NaN,Пасс (1),NaN,https://www.cian.ru/rent/flat/271766412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22989,274204547,4,м. Сокол (5 мин на машине),"Москва, Иваньковское шоссе, 5",150.0/83.0/15.0,"16/23, Монолитно-кирпичный",подземная,"250000.0 руб./ За месяц, Залог - 250000 руб., ...",Лот 81033. Ирина Волкова.\n\nБонус агенту 2500...,Евроремонт,83.0,Балкон (1),На улицу и двор,Совмещенный (2),"Можно с детьми, Можно с животными","Мебель на кухне, Ванна, Душевая кабина, Стирал...",NaN,3.0,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/274204547
23031,274776175,4,м. Щукинская (19 мин пешком),"Москва, Иваньковское шоссе, 5",150.0/100.0/20.0,16/24,NaN,"250000.0 руб./ За месяц, Залог - 250000 руб., ...",Лот 69725. Просторная светлая квартира без меб...,Евроремонт,100.0,Балкон (1),На улицу,Совмещенный (2),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Стиральная...",NaN,NaN,NaN,NaN,https://www.cian.ru/rent/flat/274776175
23132,269188642,4,м. Стрешнево (10 мин на машине),"Москва, Никольский тупик, 2к1",220.0/130.0/20.0,"4/6, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","ЭЛИТНЫЙ ЖК ЧАЙКА с круглосуточной охраной, вид...",Дизайнерский,130.0,Балкон (1),Во двор,Совмещенный (3),"Можно с детьми, Можно с животными","Мебель на кухне, Ванна, Душевая кабина, Стирал...",NaN,NaN,"Пасс (2), Груз (2)",NaN,https://www.cian.ru/rent/flat/269188642
23133,271992616,"4, Оба варианта",м. Сокол (10 мин на машине),"Москва, Береговая улица, 8К1",203.0/92.0/25.0,1/5,наземная,"600000.0 руб./ За месяц, Залог - 600000 руб., ...","ID 35735: Предлагается превосходная, просторн...",Евроремонт,92.0,Лоджия (1),На улицу,Совмещенный (2),"Можно с детьми

In [569]:
# Применяем функцию для заполнения пропусков в столбце на основе описания
def additional_description(row):
    if pd.isna(row['Дополнительно']):
        description = row['Описание'].lower()
        if any(word in description for word in ['без мебели', 'мебели нет']):
            if any(word in description for word in ['стиральная', 'техникой', 'холодильник', 'оборудована']):
                return 'мебель и техника'  # Обновляем значение на "мебель и техника"
            else:
                return 'ничего'  # Обновляем значение на "ничего"
        elif any(word in description for word in ['частично мебел', 'вся мебель', 'мебель в', 'необходимая', 'необходимой', 'меблирован', 'новой мебелью']):
            if any(word in description for word in ['стиральная', 'техникой', 'холодильник', 'оборудована']):
                return 'мебель и техника'  # Обновляем значение на "мебель и техника"
            else:
                return 'мебель'  # Обновляем значение на "мебель"
        else:
            return 'мебель и техника'
    else:
        return row['Дополнительно']

df['Дополнительно'] = df.apply(additional_description, axis=1)

In [ ]:
# #Приведение названий столбцов к нужномуу формату - английский язык + соединение слов _РЕШИЛИ ДЕЛАТЬ В КОНЦЕ,,,
df.rename(columns = {'Площадь комнат, м2': 'rooms_square', 'Балкон': 'balcony', 'Окна': 'windows', 'Санузел': 'bathroom', 'Можно с детьми/животными': 'сhildren/pets_allowed',
                    'Дополнительно': 'additionally'}, inplace=True)